# 教師なし学習実践論（寺口担当分）
注）この資料は、滋賀大学データサイエンス研究科の教師なし学習の教材です。第三者に公開することのないようお願いします。

# igraphパッケージによるネットワーク解析
-   igraphは代表的なネットワーク解析のパッケージ
-   内部はCで書かれており、高速
-   RとPythonのどちらからも利用可能

## pythonのigraphパッケージ
- RのigraphのインターフェースとPythonのighraphのインターフェースはかなり違う。
- https://python.igraph.org/en/stable/index.html
- 今回のスクリプトは、igraph v0.11.6で動作確認
- 昨年度の資料では、バックエンドとしてmatplotlibを利用していたが、今回からCairo(pycairoパッケージ)に変更

In [ ]:
# !pip install igraph
# !pip install pycairo

In [ ]:
import igraph as ig
import numpy as np
import seaborn as sns
import random

In [ ]:
ig.__version__

In [ ]:
# pycairoをインストールしていないとエラーになる。
g = ig.Graph.Famous("Zachary")
ig.plot(g)

In [ ]:
# # matplotlibをバックエンドに使う場合
# import matplotlib.pyplot as plt
# %matplotlib inline
# # DPIを設定
# plt.rcParams["figure.dpi"] = 200
# # 図のデフォルトのサイズを設定
# plt.rcParams["figure.figsize"] = (3,3)

# g = ig.Graph.Famous("Zachary")
# fig, ax = plt.subplots()
# ig.plot(g,target= ax)

In [ ]:
# igraphオブジェクト
print(g)

# 頂点の数
print("頂点の数：{}".format(g.vcount()))

# 辺の数
print("辺の数：{}".format(g.ecount()))

In [ ]:
# このグラフの頂点集合
g.vs

In [ ]:
# このグラフの辺集合
g.es

## グラフの可視化

In [ ]:
g = ig.Graph.Famous("Zachary")
ig.plot(g,vertex_color='skyblue', vertex_label=g.vs.indices,vertex_frame_width=0)

### グラフの配置

In [ ]:
# Fruchterman-Reingold配置アルゴリズム
layout = g.layout_fruchterman_reingold()
ig.plot(g,layout=layout)

In [ ]:
# 円環上に配置
layout = g.layout_circle()
ig.plot(g,layout=layout)

## グラフの作成

### 隣接行列からグラフを作成

In [ ]:
# デフォルトでは有向グラフ
g = ig.Graph.Adjacency([[0,0,0],[1,0,0],[1,1,0]])
ig.plot(g, vertex_label=g.vs.indices)

In [ ]:
# 無向グラフの時は、mode="undirected"
g = ig.Graph.Adjacency([[0,1,1],[1,0,1],[1,1,0]],mode="undirected")
ig.plot(g,vertex_label=g.vs.indices)

### 辺リストからグラフを作成

In [ ]:
# 最初に使ったグラフから辺リストを取得
g = ig.Graph.Famous("Zachary")
el = g.get_edgelist()
print(el)

In [ ]:
# 辺リストからグラフを作成
g = ig.Graph.TupleList(el)
ig.plot(g,vertex_label=g.vs.indices)

### ネットワークの生成モデル

In [ ]:
# Erdos-Renyi model
random.seed(1234)
g = ig.Graph.Erdos_Renyi(100,0.1)
ig.plot(g)

In [ ]:
# Watts-Strogatz model
random.seed(1234)
g = ig.Graph.Watts_Strogatz(dim=1,size=100,nei=2,p=0.1)
ig.plot(g)

In [ ]:
# Scale Free Network
random.seed(1234)
g = ig.Graph.Barabasi(n=100,power=1,directed=False)
ig.plot(g)

## 相関ネットワーク

In [ ]:
# ここでは、irisデータを使う
iris = sns.load_dataset('iris')

# irisデータ全体の特徴量間の相関行列
rho = iris.drop("species",axis=1).corr()

rho

In [ ]:
# 隣接行列を作成
a = rho > 0.5 # 相関係数が0.5以上を残す。
np.fill_diagonal(a.values,0) # 対角要素を0に
a

In [ ]:
# 隣接行列からグラフを作成
g = ig.Graph.Adjacency(a,mode="undirected")
ig.plot(g,vertex_label=a.index)

## 中心性指標

In [ ]:
# 最初に使ったグラフ
g = ig.Graph.Famous("Zachary") 
ig.plot(g,vertex_label=g.vs.indices)

In [ ]:
# 次数中心性
k = g.degree()
print(k)

In [ ]:
v_size=np.array(k)
ig.plot(g,vertex_size=v_size,vertex_label=g.vs.indices)

In [ ]:
# 媒介中心性
b = g.betweenness()
print(b)

In [ ]:
# 例えば、対数変換してみた例
v_size=np.log(np.array(b)+1)*5
ig.plot(g,vertex_size=v_size,vertex_label=g.vs.indices)

In [ ]:
# 辺に対する媒介中心性
b = g.edge_betweenness()
e_width=np.log(np.array(b)+1)
ig.plot(g,edge_width=e_width,vertex_label=g.vs.indices)

## コミュニティ検出

In [ ]:
# 媒介中心性を利用したコミュニティ検出
res = g.community_edge_betweenness()
communities = res.as_clustering()
num_communities = len(communities)
ig.plot(communities,vertex_label=g.vs.indices)

In [ ]:
# Louvain法
communities = g.community_multilevel()
num_communities = len(communities)
ig.plot(communities,vertex_label=g.vs.indices)

In [ ]:
# Leiden法
communities = g.community_leiden()
ig.plot(communities,vertex_label=g.vs.indices)

In [ ]:
# Leiden法
communities = g.community_leiden(resolution=0.1)
ig.plot(communities,vertex_label=g.vs.indices)

In [ ]:
# KONECTのZebraネットワークをダウンロード
# http://konect.cc/networks/
import pandas as pd
el = pd.read_table("out.moreno_zebra_zebra", comment = '%', sep=r"\s+", header=None, dtype=int) # sep=r"\s+" で、空白区切りとタブ区切りに両対応
el

In [ ]:
# 辺リストからグラフを作成
g = ig.Graph.TupleList(el.values)
ig.plot(g,vertex_label=g.vs.indices)